In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [ ]:
xls = pd.ExcelFile('./EZStacking_config.ods', engine="odf")
meta_package = pd.read_excel(xls, 'meta_package')
package_source = pd.read_excel(xls, 'package_source')
package = pd.read_excel(xls, 'package')
document = pd.read_excel(xls, 'document')

In [ ]:
package_source_type = 'full'
data_size = 'large'
kaggle = True

In [ ]:
pd_pk_import = package_source[(package_source.package_source_type   == package_source_type) &\
                              ((package_source.package_source_kaggle == 'both') | \
                               (package_source.package_source_kaggle == kaggle))\
                             ].merge(meta_package[(meta_package.meta_package_valid == True) & \
                                     ((meta_package.meta_package_data_size == 'both') | \
                                     (meta_package.meta_package_data_size == data_size))], \
                                     left_on  = 'meta_package_index', \
                                     right_on = 'meta_package_index', \
                                     how = 'inner') \
                             [['package_source_index', 'package_source_name', \
                               'package_source_short_name', 'package_source_code']]
pd_pk_import

In [ ]:
package_source_type = 'partial'
data_size = 'small'

In [ ]:
pd_pk_srce_from = package_source[(package_source.package_source_type == package_source_type)\
                                ].merge(meta_package[(meta_package.meta_package_valid == True) & \
                                         ((meta_package.meta_package_data_size == 'both') | \
                                         (meta_package.meta_package_data_size == data_size))], \
                                        left_on  = 'meta_package_index', \
                                        right_on = 'meta_package_index', \
                                        how = 'inner') \
                                [['package_source_index', 'package_source_name', \
                                  'package_source_short_name', 'package_source_code']] 
pd_pk_srce_from

In [ ]:
problem = 'regression'

In [ ]:
pd_pk_from = package[((package.package_problem == 'None') | (package.package_problem == problem)) \
                    ].merge(pd_pk_srce_from, \
                            left_on='package_source_index', \
                            right_on='package_source_index',\
                            how='inner') \
                     .merge(meta_package[(meta_package.meta_package_valid == True) & \
                                         ((meta_package.meta_package_data_size == 'both') | \
                                         (meta_package.meta_package_data_size == data_size))], \
                            left_on  = 'meta_package_index', \
                            right_on = 'meta_package_index', \
                            how = 'inner') \
                [['package_source_index', 'package_source_name', 'package_name']].drop_duplicates()
pd_pk_from

In [ ]:
pd_level_0 = package[(package.package_type == 'estimator') & \
                     (package.package_problem == problem) & \
                     (package.meta_package_index != 'STACK') \
                    ].merge(pd_pk_from, \
                            left_on='package_source_index', \
                            right_on='package_source_index',\
                            how='inner') \
                     .merge(meta_package[(meta_package.meta_package_valid == True) & \
                                         ((meta_package.meta_package_data_size == 'both') | \
                                         (meta_package.meta_package_data_size == data_size))], \
                            left_on  = 'meta_package_index', \
                            right_on = 'meta_package_index', \
                            how = 'inner') \
                    [[ 'package_index', 'package_code', 'meta_package_tree']].drop_duplicates()
pd_level_0

In [ ]:
pd_tree = package[(package.package_type == 'estimator') & \
                  (package.package_problem == problem) & \
                  (package.meta_package_index != 'STACK') \
                 ].merge(pd_pk_from, \
                         left_on='package_source_index', \
                         right_on='package_source_index',\
                         how='inner') \
                  .merge(meta_package[(meta_package.meta_package_tree == True) & \
                                      (meta_package.meta_package_valid == True) & \
                                      ((meta_package.meta_package_data_size == 'both') | \
                                      (meta_package.meta_package_data_size == data_size))], \
                         left_on  = 'meta_package_index', \
                         right_on = 'meta_package_index', \
                         how = 'inner') \
                  [[ 'package_index', 'package_code']].drop_duplicates()
pd_tree

In [ ]:
pd_document = document[((document.document_problem == 'both') | (document.document_problem == problem)) & \
                       ((document.document_data_size == 'both') | (document.document_data_size == data_size)) & \
                       ((document.document_stacking == 'both') | \
                       (document.document_stacking == \
                        meta_package[meta_package.meta_package_index == 'STACK']\
                        ['meta_package_valid'].tolist()[0]\
                       )) & \
                       ((document.document_keras == 'both') | \
                       (document.document_keras == \
                        meta_package[meta_package.meta_package_index == 'KER']\
                        ['meta_package_valid'].tolist()[0]\
                       )) & \
                       ((document.document_pipeline == 'both') | \
                       (document.document_pipeline == \
                        meta_package[meta_package.meta_package_index == 'PIP']\
                        ['meta_package_valid'].tolist()[0]\
                       )) & \
                       ((document.document_yb == 'both') | \
                       (document.document_yb == \
                        meta_package[meta_package.meta_package_index == 'YB']\
                        ['meta_package_valid'].tolist()[0]\
                       )) \
                      ].merge(meta_package[meta_package.meta_package_valid == True], \
                                           left_on  = 'meta_package_index', \
                                           right_on = 'meta_package_index', \
                                           how = 'inner') \
                      [['sort_key', 'title', 'text', 'code']].sort_values('sort_key') \
                      [['title', 'text', 'code']].drop_duplicates()

pd_document